In [ ]:
%sql -- Extração de ordens (AUFK) 

WITH ordens_filtradas AS (
    SELECT aufk.DI_SEQUENCE_NUMBER, aufk.AUFNR, aufk.MANDT
    FROM poseidon_uc.saps4hanarawview.aufk AS aufk
    WHERE aufk.AUART IN ('GIA','GAA','GRA') 
    AND aufk.BUKRS IN ('GHIN', 'GHSM', 'EPEI')
    -- AND aufk.BUKRS IN ('GHIN')
    AND aufk.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.aufk AS a
        WHERE a.AUFNR = aufk.AUFNR
    )
    AND aufk.DI_OPERATION_TYPE != 'D'
    LIMIT 10  -- Processar em lotes
)
SELECT
    aufk.*
    -- aufk.AUFNR AS ordem,
    -- aufk.AUART AS tipo_ordem,
    -- aufk.BUKRS AS empresa,
    -- aufk.KTEXT AS texto_breve,
    -- aufk.ERDAT AS data_inicio_real,
    -- aufk.WERKS AS centro_trabalho,
    -- aufk.VAPLZ AS centro_trabalho_princ

FROM ordens_filtradas of
INNER JOIN poseidon_uc.saps4hanarawview.aufk AS aufk
    ON aufk.AUFNR = of.AUFNR 
    AND aufk.DI_SEQUENCE_NUMBER = of.DI_SEQUENCE_NUMBER
    AND aufk.MANDT = of.MANDT

ORDER BY aufk.AUFNR

In [ ]:
%sql -- Extração enriquecida de ordens (AUFK) com a tabela AFIH

WITH ordens_filtradas AS (
    SELECT aufk.DI_SEQUENCE_NUMBER, aufk.AUFNR, aufk.MANDT
    FROM poseidon_uc.saps4hanarawview.aufk AS aufk
    WHERE aufk.AUART IN ('GIA','GAA','GRA') 
    -- AND aufk.BUKRS IN ('GHIN', 'GHSM', 'EPEI')
    AND aufk.BUKRS IN ('GHIN')
    AND aufk.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.aufk AS a
        WHERE a.AUFNR = aufk.AUFNR
    )
    AND aufk.DI_OPERATION_TYPE != 'D'
    LIMIT 10  -- Processar em lotes
),
afih_filtrado AS (
    -- SELECT afih.DI_SEQUENCE_NUMBER, afih.MANDT, afih.AUFNR, afih.PRIOK, afih.ARTPR
    SELECT afih.*
    FROM poseidon_uc.saps4hanarawview.afih AS afih
    WHERE afih.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.afih AS a
        WHERE a.MANDT = afih.MANDT 
        AND a.AUFNR = afih.AUFNR
    )
)
SELECT
    aufk.AUFNR AS ordem,
    -- aufk.AUART AS tipo_ordem,
    -- aufk.BUKRS AS empresa,
    -- aufk.KTEXT AS texto_breve,
    -- aufk.ERDAT AS data_criacao,
    -- aufk.WERKS AS centro_trabalho,
    
    afih.*,

    t356_t.PRIOKX AS texto_prioridade

FROM ordens_filtradas of
INNER JOIN poseidon_uc.saps4hanarawview.aufk AS aufk
    ON aufk.AUFNR = of.AUFNR 
    AND aufk.DI_SEQUENCE_NUMBER = of.DI_SEQUENCE_NUMBER
    AND aufk.MANDT = of.MANDT

-- LEFT JOIN poseidon_uc.saps4hanarawview.afko AS afko
--     ON afko.MANDT = aufk.MANDT AND afko.AUFNR = aufk.AUFNR

LEFT JOIN afih_filtrado AS afih
    ON afih.MANDT = aufk.MANDT AND afih.AUFNR = aufk.AUFNR

LEFT JOIN poseidon_uc.saps4hanarawview.t356_t AS t356_t
    ON t356_t.PRIOK = afih.PRIOK
    AND t356_t.ARTPR = afih.ARTPR
    AND t356_t.SPRAS = 'P'

ORDER BY aufk.AUFNR

In [ ]:
%sql -- Extração enriquecida de ordens (AUFK) com a tabela AFKO

WITH ordens_filtradas AS (
    SELECT aufk.DI_SEQUENCE_NUMBER, aufk.AUFNR, aufk.MANDT
    FROM poseidon_uc.saps4hanarawview.aufk AS aufk
    WHERE aufk.AUART IN ('GIA','GAA','GRA') 
    AND aufk.BUKRS IN ('GHIN', 'GHSM', 'EPEI')
    -- AND aufk.BUKRS IN ('GHIN')
    AND aufk.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.aufk AS a
        WHERE a.AUFNR = aufk.AUFNR
    )
    AND aufk.DI_OPERATION_TYPE != 'D'
    LIMIT 10  -- Processar em lotes
),
afko_filtrado AS (
    -- SELECT afih.DI_SEQUENCE_NUMBER, afih.MANDT, afih.AUFNR, afih.PRIOK, afih.ARTPR
    SELECT afko.*
    FROM poseidon_uc.saps4hanarawview.afko AS afko
    WHERE afko.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.afko AS a
        WHERE a.MANDT = afko.MANDT 
        AND a.AUFNR = afko.AUFNR
    )
)
SELECT
    aufk.AUFNR AS ordem,
    -- aufk.AUART AS tipo_ordem,
    -- aufk.BUKRS AS empresa,
    -- aufk.KTEXT AS texto_breve,
    -- aufk.ERDAT AS data_criacao,
    -- aufk.WERKS AS centro_trabalho,
    
    afko.*

FROM ordens_filtradas of
INNER JOIN poseidon_uc.saps4hanarawview.aufk AS aufk
    ON aufk.AUFNR = of.AUFNR 
    AND aufk.DI_SEQUENCE_NUMBER = of.DI_SEQUENCE_NUMBER
    AND aufk.MANDT = of.MANDT

LEFT JOIN afko_filtrado AS afko
    ON afko.MANDT = aufk.MANDT AND afko.AUFNR = aufk.AUFNR

-- WHERE

ORDER BY aufk.AUFNR
-- LIMIT 10

In [ ]:
%sql -- Extração enriquecida de ordens (AUFK) com a tabela AFRU

WITH ordens_filtradas AS (
    SELECT aufk.DI_SEQUENCE_NUMBER, aufk.AUFNR, aufk.MANDT
    FROM poseidon_uc.saps4hanarawview.aufk AS aufk
    WHERE aufk.AUART IN ('GIA','GAA','GRA') 
    AND aufk.BUKRS IN ('GHIN', 'GHSM', 'EPEI')
    -- AND aufk.BUKRS IN ('GHIN')
    AND aufk.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.aufk AS a
        WHERE a.AUFNR = aufk.AUFNR
    )
    AND aufk.DI_OPERATION_TYPE != 'D'
    LIMIT 10  -- Processar em lotes
),
afru_filtrado AS (
    -- SELECT afih.DI_SEQUENCE_NUMBER, afih.MANDT, afih.AUFNR, afih.PRIOK, afih.ARTPR
    SELECT afru.*
    FROM poseidon_uc.saps4hanarawview.afru AS afru
    WHERE afru.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.afru AS a
        WHERE a.MANDT = afru.MANDT 
        AND a.AUFNR = afru.AUFNR
    )
)
SELECT
    aufk.AUFNR AS ordem,
    -- aufk.AUART AS tipo_ordem,
    -- aufk.BUKRS AS empresa,
    -- aufk.KTEXT AS texto_breve,
    -- aufk.ERDAT AS data_criacao,
    -- aufk.WERKS AS centro_trabalho,
    
    afru.*

FROM ordens_filtradas of
INNER JOIN poseidon_uc.saps4hanarawview.aufk AS aufk
    ON aufk.AUFNR = of.AUFNR 
    AND aufk.DI_SEQUENCE_NUMBER = of.DI_SEQUENCE_NUMBER
    AND aufk.MANDT = of.MANDT

LEFT JOIN afru_filtrado AS afru
    ON afru.MANDT = aufk.MANDT AND afru.AUFNR = aufk.AUFNR

-- WHERE

ORDER BY aufk.AUFNR
-- LIMIT 10

In [ ]:
%sql -- Extração enriquecida de ordens (AUFK) com a tabela AFVC
-- afvc completa (sem filtro) pra ordens específicas

WITH ordens_filtradas AS (
    SELECT aufk.DI_SEQUENCE_NUMBER, aufk.AUFNR, aufk.MANDT
    FROM poseidon_uc.saps4hanarawview.aufk AS aufk
    WHERE aufk.AUART IN ('GIA','GAA','GRA') 
    AND aufk.BUKRS IN ('GHIN', 'GHSM', 'EPEI')
    -- AND aufk.BUKRS IN ('GHIN')
    AND aufk.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.aufk AS a
        WHERE a.AUFNR = aufk.AUFNR
    )
    AND aufk.DI_OPERATION_TYPE != 'D'
    AND ( -- retorno nulo para essas ordens, tentar alternativa com a tabela afru
        aufk.AUFNR = 610000042263 OR
        aufk.AUFNR = 610000042264 OR
        aufk.AUFNR = 610000051699 OR
        aufk.AUFNR = 610000051729 OR
        aufk.AUFNR = 610000051755 OR
        aufk.AUFNR = 610000051772 OR
        aufk.AUFNR = 610000052429 OR
        aufk.AUFNR = 610000052431
    ) 
    -- LIMIT 10  -- Processar em lotes
),
afko_filtrado AS (
    -- SELECT afih.DI_SEQUENCE_NUMBER, afih.MANDT, afih.AUFNR, afih.PRIOK, afih.ARTPR
    SELECT afko.*
    FROM poseidon_uc.saps4hanarawview.afko AS afko
    WHERE afko.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.afko AS a
        WHERE a.MANDT = afko.MANDT 
        AND a.AUFNR = afko.AUFNR
    )
)
-- ,
-- afvc_filtrado AS (
--     SELECT afvc.*
--     FROM poseidon_uc.saps4hanarawview.afvc AS afvc
--     WHERE afvc.VORNR = '0010'
--     AND afvc.DI_SEQUENCE_NUMBER = (
--         SELECT MAX(a.DI_SEQUENCE_NUMBER)
--         FROM poseidon_uc.saps4hanarawview.afvc AS a
--         WHERE a.MANDT = afvc.MANDT
--         AND a.AUFPL = afvc.AUFPL
--         AND a.VORNR = '0010'
--     )
-- )
SELECT
    aufk.AUFNR AS ordem,
    -- aufk.AUART AS tipo_ordem,
    -- aufk.BUKRS AS empresa,
    -- aufk.KTEXT AS texto_breve,
    -- aufk.ERDAT AS data_criacao,
    -- aufk.WERKS AS centro_trabalho,
    
    afvc.*

FROM ordens_filtradas of
INNER JOIN poseidon_uc.saps4hanarawview.aufk AS aufk
    ON aufk.AUFNR = of.AUFNR 
    AND aufk.DI_SEQUENCE_NUMBER = of.DI_SEQUENCE_NUMBER
    AND aufk.MANDT = of.MANDT

LEFT JOIN afko_filtrado AS afko
    ON afko.MANDT = aufk.MANDT AND afko.AUFNR = aufk.AUFNR

LEFT JOIN poseidon_uc.saps4hanarawview.afvc AS afvc
    ON afvc.MANDT = afko.MANDT AND afvc.AUFPL = afko.AUFPL

-- WHERE

ORDER BY aufk.AUFNR
-- LIMIT 10

In [ ]:
%sql -- Extração enriquecida de ordens (AUFK) com a tabela AFVC


WITH ordens_filtradas AS (
    SELECT aufk.DI_SEQUENCE_NUMBER, aufk.AUFNR, aufk.MANDT
    FROM poseidon_uc.saps4hanarawview.aufk AS aufk
    WHERE aufk.AUART IN ('GIA','GAA','GRA') 
    AND aufk.BUKRS IN ('GHIN', 'GHSM', 'EPEI')
    -- AND aufk.BUKRS IN ('GHIN')
    AND aufk.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.aufk AS a
        WHERE a.AUFNR = aufk.AUFNR
    )
    AND aufk.DI_OPERATION_TYPE != 'D'
    LIMIT 10  -- Processar em lotes
),
afko_filtrado AS (
    -- SELECT afih.DI_SEQUENCE_NUMBER, afih.MANDT, afih.AUFNR, afih.PRIOK, afih.ARTPR
    SELECT afko.*
    FROM poseidon_uc.saps4hanarawview.afko AS afko
    WHERE afko.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.afko AS a
        WHERE a.MANDT = afko.MANDT 
        AND a.AUFNR = afko.AUFNR
    )
),
afvc_filtrado AS (
    SELECT afvc.*
    FROM poseidon_uc.saps4hanarawview.afvc AS afvc
    WHERE afvc.VORNR IN ('0010', '010', '10')
    AND afvc.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.afvc AS a
        WHERE a.MANDT = afvc.MANDT
        AND a.AUFPL = afvc.AUFPL
        AND a.VORNR IN ('0010', '010', '10')
    )
)
SELECT
    aufk.AUFNR AS ordem,
    -- aufk.AUART AS tipo_ordem,
    -- aufk.BUKRS AS empresa,
    -- aufk.KTEXT AS texto_breve,
    -- aufk.ERDAT AS data_criacao,
    -- aufk.WERKS AS centro_trabalho,
    
    afvc.*

FROM ordens_filtradas of
INNER JOIN poseidon_uc.saps4hanarawview.aufk AS aufk
    ON aufk.AUFNR = of.AUFNR 
    AND aufk.DI_SEQUENCE_NUMBER = of.DI_SEQUENCE_NUMBER
    AND aufk.MANDT = of.MANDT

LEFT JOIN afko_filtrado AS afko
    ON afko.MANDT = aufk.MANDT AND afko.AUFNR = aufk.AUFNR

LEFT JOIN afvc_filtrado AS afvc
    ON afvc.MANDT = afko.MANDT AND afvc.AUFPL = afko.AUFPL

-- WHERE afvc.DI_SEQUENCE_NUMBER IS NULL

ORDER BY aufk.AUFNR
-- LIMIT 10

In [ ]:
%sql -- Extração enriquecida de ordens (AUFK) com a tabela ILOA (diretamente)
-- Retorno nulo, nao funciona

WITH ordens_filtradas AS (
    SELECT aufk.DI_SEQUENCE_NUMBER, aufk.AUFNR, aufk.MANDT
    FROM poseidon_uc.saps4hanarawview.aufk AS aufk
    WHERE aufk.AUART IN ('GIA','GAA','GRA') 
    AND aufk.BUKRS IN ('GHIN', 'GHSM', 'EPEI')
    -- AND aufk.BUKRS IN ('GHIN')
    AND aufk.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.aufk AS a
        WHERE a.AUFNR = aufk.AUFNR
    )
    AND aufk.DI_OPERATION_TYPE != 'D'
    LIMIT 10  -- Processar em lotes
),
iloa_filtrado AS (
    -- SELECT afih.DI_SEQUENCE_NUMBER, afih.MANDT, afih.AUFNR, afih.PRIOK, afih.ARTPR
    SELECT iloa.*
    FROM poseidon_uc.saps4hanarawview.iloa AS iloa
    WHERE iloa.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.iloa AS a
        WHERE a.MANDT = iloa.MANDT 
        AND a.AUFNR = iloa.AUFNR
    )
)
SELECT
    aufk.AUFNR AS ordem,
    -- aufk.AUART AS tipo_ordem,
    -- aufk.BUKRS AS empresa,
    -- aufk.KTEXT AS texto_breve,
    -- aufk.ERDAT AS data_criacao,
    -- aufk.WERKS AS centro_trabalho,
    
    iloa.*

FROM ordens_filtradas of
INNER JOIN poseidon_uc.saps4hanarawview.aufk AS aufk
    ON aufk.AUFNR = of.AUFNR 
    AND aufk.DI_SEQUENCE_NUMBER = of.DI_SEQUENCE_NUMBER
    AND aufk.MANDT = of.MANDT

LEFT JOIN iloa_filtrado AS iloa
    ON iloa.MANDT = aufk.MANDT AND iloa.AUFNR = aufk.AUFNR

ORDER BY aufk.AUFNR

In [ ]:
%sql -- Extração enriquecida de ordens (AUFK) com a tabela ILOA (culuna afih.iloan como intermediaria)

WITH ordens_filtradas AS (
    SELECT aufk.DI_SEQUENCE_NUMBER, aufk.AUFNR, aufk.MANDT
    FROM poseidon_uc.saps4hanarawview.aufk AS aufk
    WHERE aufk.AUART IN ('GIA','GAA','GRA') 
    AND aufk.BUKRS IN ('GHIN', 'GHSM', 'EPEI')
    -- AND aufk.BUKRS IN ('GHIN')
    AND aufk.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.aufk AS a
        WHERE a.AUFNR = aufk.AUFNR
    )
    AND aufk.DI_OPERATION_TYPE != 'D'
    LIMIT 10  -- Processar em lotes
),
afih_filtrado AS (
    -- SELECT afih.DI_SEQUENCE_NUMBER, afih.MANDT, afih.AUFNR, afih.PRIOK, afih.ARTPR
    SELECT afih.*
    FROM poseidon_uc.saps4hanarawview.afih AS afih
    WHERE afih.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.afih AS a
        WHERE a.MANDT = afih.MANDT 
        AND a.AUFNR = afih.AUFNR
    )
)
SELECT
    aufk.AUFNR AS ordem,
    -- aufk.AUART AS tipo_ordem,
    -- aufk.BUKRS AS empresa,
    -- aufk.KTEXT AS texto_breve,
    -- aufk.ERDAT AS data_criacao,
    -- aufk.WERKS AS centro_trabalho,
    afih.ILOAN AS afih_iloan,
    iloa.*

FROM ordens_filtradas of
INNER JOIN poseidon_uc.saps4hanarawview.aufk AS aufk
    ON aufk.AUFNR = of.AUFNR 
    AND aufk.DI_SEQUENCE_NUMBER = of.DI_SEQUENCE_NUMBER
    AND aufk.MANDT = of.MANDT

-- LEFT JOIN poseidon_uc.saps4hanarawview.afko AS afko
--     ON afko.MANDT = aufk.MANDT AND afko.AUFNR = aufk.AUFNR

LEFT JOIN afih_filtrado AS afih
    ON afih.MANDT = aufk.MANDT AND afih.AUFNR = aufk.AUFNR

LEFT JOIN poseidon_uc.saps4hanarawview.iloa AS iloa
    ON iloa.MANDT = afih.MANDT AND iloa.ILOAN = afih.ILOAN

-- WHERE

ORDER BY aufk.AUFNR